# 신경망 학습

학습이란 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 흭득하는 것을 의미한다.

기계학습 문제는 데이터를 **훈련 데이터(training data)**와 **시험 데이터(test data)**로 나눠 학습과 시험을 수행하는 것이 일반적입니다.

보통의 진행은 훈련 데이터만 사용하여 학습하면서 최적의 매개변수를 찾습니다. 그 후 시험 데이터를 사용하여 앞서 훈련한 모델의 실력을 평가합니다.

이를 하는 이유는 **범용 능력**을 제대로 평가하기 위함입니다.
**범용 능력** : 학습 하지 못한 데이터도 올바르게 푸는 능력

하지만 그렇다고 한 데이터셋으로만 지나치게 최적화 되면 **오버피팅**이라는 문제가 생깁니다.

## 손실함수

성능의 '나쁨'을 나타내는 지표로 일반적으로 평군 제곱 오차와 교차 엔트로피 오차를 사용

### 평균 제곱 오차  

E = 1/2 * (sum(yk -tk) ** 2)  
yk = 신경망의 출력  
tk = 정답 레이블  
k는 데이터의 차원 수를 나타냄

In [4]:
import numpy as np

In [5]:
def mean_squared_error(y, k):
    return 0.5 * np.sum((y-k)**2)

In [6]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

In [8]:
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

### 교차 엔트로피 오차  
  
E = -(sum(tk * log yk))  
yk = 신경망의 출력  
tk = 정답 레이블(one-hot-encoding)

사실상 정답 레이일 때로 추정되면 자연로그를 계산하는 방식

In [9]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

아주 작은 값인 delta를 더하는 이유는 np.log()에 0을 입력하면 -inf가 출력된다.  
이를 방지하기 위함

In [10]:
np.log(0)

<ipython-input-10-f6e7c0610b57>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [12]:
cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [14]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

cross_entropy_error(np.array(y),np.array(t))

2.302584092994546

정답일 시 출력되는 값이 더 낮다.  
기계학습은 훈련 데이터에 대한 손실 함수의 값을 구해서 매개변수를 정정한다.  
만약 100개의 훈련 데이터가 존재 한다면 각각 손실 함수를 구한 후 그 값의 합을 지표로 삼는다.

### 손실 함수의 합

교차 엔트로피 오차의 합 식  
E = -(1/N * sum(sum(tnk * log ynk)))  

N = 데이터의 갯수  
tnk = n번째 데이터의 k번째 값

### 미니배치

모든 데이터의 손실함수 값의 합을 구하는 것은 힘든다.(배치의 양이 너무 크다.)  
그렇기 때문에 훈련 데이터의 일부만 골라 추정치를 구한다.(일부만 뽑는다.)

MNIST를 이용한 미니배치 학습 예제

In [15]:
import sys, os
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_size = x_train.shape[0] # 60000
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 트레이닝 데이터 중 batch_size만큼을 뽑는다.
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

데이터 하나당 교차 엔트로피 오차를 구하는 교차 엔트로피 오차 구현

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = t.reshape(1, y.size)
        
    batch_size = y.shpae[0]
    return -np.sum(np.log(y[np.arang(batch_size), t] + 1e - 7)) / batch_size

## 손실함수를 사용하는 이유  
  
정확도나 계단 함수를 사용하면 가중치의 작은 변화에 따른 연속적인 변화가 나타나지 않는다.  
하지만 시그모이드 함수와 같은 손실 함수는 작은 가중치 변화에도 연속적인 변화가 나타난ㄷ.  
시그모이드의 함수의 미분은 어느 장소라도 0이 되지는 않는다.